In [1]:
import os
import PyPDF2
import json

In [2]:
def is_scanned_pdf(file_path):
    with open(file_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        
        for page_num in range(len(reader.pages)):
            page = reader.pages[page_num]
            text = page.extract_text()
            if text and text.strip():  
                return False
    return True

In [4]:
def parse_pdf_to_json(file_path):
    result = []
    file_name = os.path.basename(file_path)
    
    with open(file_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        
        for page_num in range(len(reader.pages)):
            page = reader.pages[page_num]
            text = page.extract_text()

            if text and text.strip():
                page_data = {
                    "raw_text": text.strip(),
                    "file_path": file_path,
                    "doc_type": "text",
                    "filename": file_name,
                    "page_number": page_num + 1
                }
                result.append(page_data)

    return result


In [6]:

# Пример использования

file_path = r".\Данные Хакатон ИТМО\ГОСТ 5781-82.pdf"

if not is_scanned_pdf(file_path):
    parsed_data = parse_pdf_to_json(file_path)
    
    json_output = json.dumps(parsed_data, ensure_ascii=False, indent=4)

    print(json_output)

    # Сохраняем результат в файл, если нужно:
    # with open("parsed_output.json", "w", encoding="utf-8") as json_file:
    #     json_file.write(json_output)

[
    {
        "raw_text": "ГОСТ 5781-82\nМЕЖГОСУДАРСТВЕННЫЙ СТАНДАРТ\nСТАЛЬ ГОРЯЧЕКАТАНАЯ  \nДЛЯ АРМИРОВАНИЯ  \nЖЕЛЕЗОБЕТОННЫХ КОНСТРУКЦИЙ\nТЕХНИЧЕСКИЕ УСЛОВИЯ\nИздание официальное\nСтандартинформ\n2006\nболеро спицами",
        "file_path": ".\\Данные Хакатон ИТМО\\ГОСТ 5781-82.pdf",
        "doc_type": "text",
        "filename": "ГОСТ 5781-82.pdf",
        "page_number": 1
    },
    {
        "raw_text": "УДК 6 6 4 .1 4 -1 2 2 .4 .6 9 1 .8 7 .0 0 6 .3 5 4 Группа В22\nМЕЖГОСУДАРСТВЕННЫЙ СТАНДАРТ\nСТАЛЬ ГОРЯЧЕКАТАНАЯ ДЛЯ АРМИРОВАНИЯ  \nЖЕЛЕЗОБЕТОННЫХ КОНСТРУКЦИЙ\nТехнические условияГО С Т\n5 7 8 1 - 8 2\nHot-rolled steel for reinforcement of ferroconcrete  \nstructures. Specifications\nMKC 77.140.15\nОКП 09 3004; 09 3005; 09 3006; 09 3007; 09 3008\nДата введения 01.07.83\nНастоящий стандарт распространяется на горячекатаную круглую сталь гладкого и периоди­\nческого профиля, предназначенную для армирования обычных и предварительно напряженных \nжелезобетонных конструкций (арматурна